In [1]:
import qcodes as qc
import numpy as np

from time import sleep

from qcodes.instrument_drivers.QDev.QDac import QDac

In [2]:
# Connect to the instrument
qdac = QDac('qdac', 'ASRL2::INSTR')

Connected to QDac on ASRL2::INSTR, Software Version: 0.160915
[*] Querying all channels for voltages and currents...
[+] Done


# Basic QDac Usage

The QCoDeS QDac driver currently supports using
  * 48 Output Channels
  * 8 Function generators

Each output channel has seven parameters:
  * DC voltage
  * DC voltage range
  * Current out
  * Current our range
  * Amplitude 
  * Offset
  * Associated function generator

The amplitude and offset are applied to the basic function shape of the function generator bound to the channel (when not DC).

Each function generator has four parameters:
  * Wave type
  * Frequency
  * Duty cycle (not relevant to sine)
  * No. of repetitions

In [3]:
# As an example, let's make a 50 Hz sine wave with 200 repetitions
# and output it on channel 1 and 2 with different amplitudes and offsets

# first, we select function generator 1 to output the desired waveform
qdac.fungen1_type('SINE')
qdac.fungen1_freq(50)
qdac.fungen1_nreps(200)

# Then we bind the function generator to channel 1 and 2 and set ampl. and offset
qdac.ch01_fungen(1)
qdac.ch01_ampl(0.1)
qdac.ch01_offset(0.4)
qdac.ch02_fungen(1)
qdac.ch02_ampl(-0.2)
qdac.ch02_offset(0)

# And the finally start outputting
#(Note: this is not synchronous across several generators as 
# the hardware only allows us to switch on the outputs sequentially)

qdac.run(1)  # run function generator 1

1 [1, 2]


In [4]:
# Note: we can change the DC output of channel 1 as we wish...
qdac.ch01_v(0.1)
sleep(1)
qdac.ch01_v(0.5)

In [7]:
#...but it is still bound to function generator 1:
qdac.run(1)

1 [2]


In [6]:
# if this is undesired, we may bind it back to DC:
qdac.ch01_fungen(0)

In [ ]:
# now running function generator 1 will only output on channel 2
qdac.run(1)

# Overview of channel settings

The driver provides a method for pretty-printing the state of all channels. On startup, all channels are queried for voltage and current across them, but the current query is very slow (blame the hardware).

The pretty-print method may or may not **update** the values for the currents.

In [8]:
qdac.print_overview(update_currents=True)

Channel 1 
    Current: -160.292709 (muA). Voltage: 0.39999 (V). 
    Current range: 100 muA. Voltage range: -10 V to 10 V. 
Channel 2 
    Current: -160.292709 (muA). Voltage: 0.0 (V). 
    Current range: 100 muA. Voltage range: -10 V to 10 V. 
Channel 3 
    Current: -160.292709 (muA). Voltage: 0.0 (V). 
    Current range: 100 muA. Voltage range: -10 V to 10 V. 
Channel 4 
    Current: -160.292709 (muA). Voltage: 0.0 (V). 
    Current range: 100 muA. Voltage range: -10 V to 10 V. 
Channel 5 
    Current: -160.292709 (muA). Voltage: 0.0 (V). 
    Current range: 100 muA. Voltage range: -10 V to 10 V. 
Channel 6 
    Current: -160.292709 (muA). Voltage: 0.0 (V). 
    Current range: 100 muA. Voltage range: -10 V to 10 V. 
Channel 7 
    Current: -160.292709 (muA). Voltage: 0.0 (V). 
    Current range: 100 muA. Voltage range: -10 V to 10 V. 
Channel 8 
    Current: -160.292709 (muA). Voltage: 0.0 (V). 
    Current range: 100 muA. Voltage range: -10 V to 10 V. 
Channel 9 
    Current: 5.49

## Voltage ranges

The Voltage range of each channel should be understood as a multiplier (voltage times 0.1) that can either be ON or OFF.

In [15]:
# This outputs 0.2 volts
qdac.ch01_vrange(1)
qdac.ch01_v(2)
# This also outputs 0.2 volts
qdac.ch02_fungen(0)
qdac.ch02_vrange(0)
qdac.ch02_v(0.2)

In [16]:
qdac.close()